In [61]:
library(stringr)
library(dplyr)
library(tidyr)

use read.delim to read tab separated files and read.csv for comma separated files

In [62]:
companies <- read.delim('companies.txt', stringsAsFactors = F)
rounds2 <- read.csv('rounds2.csv', stringsAsFactors = F)

In [63]:
head(companies)

permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,
/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network Media,operating,USA,DE,DE - Other,Delaware City,04-09-2014
/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,,,,,
/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010
/Organization/01Games-Technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,,Hong Kong,Hong Kong,


In [64]:
head(rounds2)

company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000
/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NA
/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,,01-03-2014,700000
/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878
/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000
/ORGANIZATION/004-TECHNOLOGIES,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,,24-07-2014,NA


unique companies present in companies data frame

In [65]:
companies$permalink <- tolower(companies$permalink)
n_distinct(companies$permalink)

[1] 66368

unique companies present in rounds2 data frame

In [66]:
rounds2$company_permalink <- tolower(rounds2$company_permalink)
n_distinct(rounds2$company_permalink)

[1] 66368

anti_join from dplyr helps to know the values present in one df that is not in other  
companies in rounds2 df not in companies df

In [67]:
rounds2 %>% anti_join(companies, by = c('company_permalink' = 'permalink')) %>% count()

n
0


full_join helps in merging two dfs based on a column(s)  
merging companies and rounds2 datatables

In [68]:
master_frame <- rounds2 %>% full_join(companies, by = c('company_permalink' = 'permalink'))

funding amount grouped by funding type

In [69]:
master_frame %>% group_by(funding_round_type) %>% summarise(avg_fund = mean(raised_amount_usd, na.rm = T))

funding_round_type,avg_fund
angel,958694.5
convertible_note,1453438.5
debt_financing,17043526.0
equity_crowdfunding,538368.2
grant,4300576.3
non_equity_assistance,411203.1
post_ipo_debt,168704571.8
post_ipo_equity,82182493.9
private_equity,73308593.0
product_crowdfunding,1363131.1


top english speaking countries with highest funding

In [70]:
master_frame %>% group_by(country_code) %>% summarise(tot_fund = sum(raised_amount_usd, na.rm = T)) %>%
  arrange(desc(tot_fund)) %>% head()

country_code,tot_fund
USA,669482123821
CHN,75703565796
,57111228725
GBR,32767048060
IND,27686336560
CAN,18424675109


category_list column has multiple categories seperated by '|'  
we just need the first value

In [71]:
master_frame$category_list <- str_split_fixed(master_frame$category_list, "\\|", n = 2)[,1]

tidyr's gather function can be used to convert data from wide format to long format  
reading the mapping.csv file and converting to long format

In [72]:
mapping <- read.csv('mapping.csv', stringsAsFactors = F)
mapping <- mapping %>% gather(key = 'main_sector', value = 'status', -category_list) %>%
  filter(status != 0) %>% select(-status)

In [73]:
mapping$main_sector  <- str_replace(mapping$main_sector, "Blanks", "Others")
mapping$category_list <- tolower(mapping$category_list)
master_frame$category_list <- tolower(master_frame$category_list)

correcting the spelling errors

In [74]:
mapping$category_list <- str_replace_all(mapping$category_list, '0', 'na')
mapping[which(mapping$category_list == 'enterprise 2.na'), 'category_list'] <- 'enterprise 2.0'

In [75]:
# total categories that are present in master_frame but not in mapping
master_frame %>% anti_join(mapping, by = c('category_list')) %>% distinct(category_list) %>% count()

n
41


In [76]:
# categories that are present in mapping but not in master_frame
mapping %>% anti_join(master_frame, by = c('category_list')) %>% distinct(category_list)

category_list


In [77]:
# merging the two data frames
master_frame <- master_frame %>% left_join(mapping, by = c('category_list'))

In [78]:
# since few categories are not present in mapping after merging NAs will be generated
# in the master_frame. Replace NA with Others category
master_frame$main_sector <- str_replace_na(master_frame$main_sector, 'Others')

creating separate data frames from USA, GBR, IND and for venture funding type

In [79]:
d1 <- master_frame %>% filter(country_code == 'USA', funding_round_type == 'venture')
d2 <- master_frame %>% filter(country_code == 'GBR', funding_round_type == 'venture')
d3 <- master_frame %>% filter(country_code == 'IND', funding_round_type == 'venture')

In [80]:
# total number and amount of investements in each main sector

number_and_amount_invst <- function(d){
  d %>% group_by(main_sector) %>% summarise(tot_inv = n(), amt_inv = sum(raised_amount_usd, na.rm = T)) %>%
    arrange(desc(amt_inv)) %>% print()
}

In [81]:
number_and_amount_invst(d1)

# A tibble: 8 x 3
  main_sector                             tot_inv      amt_inv
  <chr>                                     <int>        <dbl>
1 Cleantech...Semiconductors                 8268 119931874301
2 Social..Finance..Analytics..Advertising    7822  86331402349
3 Others                                     9115  85479942169
4 News..Search.and.Messaging                 4618  45600521085
5 Health                                     3413  31675057766
6 Manufacturing                              2643  24845319470
7 Entertainment                              1927  17014560960
8 Automotive...Sports                         566  11632164696


In [82]:
number_and_amount_invst(d2)

# A tibble: 8 x 3
  main_sector                             tot_inv    amt_inv
  <chr>                                     <int>      <dbl>
1 Cleantech...Semiconductors                  466 5073290093
2 Others                                      619 4733603292
3 Social..Finance..Analytics..Advertising     481 3773328672
4 News..Search.and.Messaging                  265 2956023749
5 Entertainment                               160 1162564878
6 Health                                      126 1046654806
7 Manufacturing                               132 1019283214
8 Automotive...Sports                          54  480878712


In [83]:
number_and_amount_invst(d3)

# A tibble: 8 x 3
  main_sector                             tot_inv    amt_inv
  <chr>                                     <int>      <dbl>
1 Others                                      340 6459857440
2 News..Search.and.Messaging                  154 1739579948
3 Social..Finance..Analytics..Advertising     193 1578804920
4 Automotive...Sports                          32 1556350000
5 Entertainment                                90  976191777
6 Cleantech...Semiconductors                   63  786444633
7 Manufacturing                                64  730260000
8 Health                                       56  564370000


In [84]:
# company with highest investment in Others category
company_with_high_invst <- function(d){
  d %>% filter(main_sector == 'Others') %>% group_by(name) %>% 
    summarise(amt_inv = sum(raised_amount_usd, na.rm = T)) %>% 
    arrange(desc(amt_inv)) %>% head()
}

In [85]:
company_with_high_invst(d1)

name,amt_inv
SoFi,1365199900
Snapchat,1165100000
LivingSocial,818225039
Facebook,615200000
Zenefits,581500000
CoreExpress,573000000


In [86]:
company_with_high_invst(d2)

name,amt_inv
OneWeb,500000000
Farfetch,194500000
Liquid Telecom,150000000
picoChip,111500000
NewVoiceMedia,111300000
Six Degrees Group,106392886


In [87]:
company_with_high_invst(d3)

name,amt_inv
Flipkart,2451000000
Snapdeal,310699998
Tata Teleservices,212000000
Myntra,158750000
Pepperfry.com,128000000
Delhivery,127500000
